In [1]:
import rasterio
from rasterio.warp import reproject, Resampling
import glob, os
import numpy as np

In [2]:
era5_dir = r"C:\Users\Ankit\Datasets_Forest_fire\ERA5_fast_tif_stacks"   # input ERA5 folder
ref_file = r"C:\Users\Ankit\Datasets_Forest_fire\VIIRS_fire_time_stack.tif"  # reference VIIRS raster
out_dir = r"C:\Users\Ankit\Datasets_Forest_fire\ERA5_resampled_to_VIIRS"      # output folder
os.makedirs(out_dir, exist_ok=True)

In [3]:
# Open VIIRS reference grid
with rasterio.open(ref_file) as ref:
    ref_profile = ref.profile
    ref_transform = ref.transform
    ref_crs = ref.crs if ref.crs is not None else "EPSG:4326"  # <-- force CRS if missing
    ref_height = ref.height
    ref_width = ref.width

In [4]:
# Loop through all ERA5 tif files
import numpy as np 
for tif_file in glob.glob(os.path.join(era5_dir, "*.tif")):
    with rasterio.open(tif_file) as src:
        out_file = os.path.join(
            out_dir,
            os.path.basename(tif_file).replace(".tif", "_toVIIRS.tif")
        )

        # Prepare output profile
        profile = src.profile.copy()
        profile.update(
            driver="GTiff",
            height=ref_height,
            width=ref_width,
            crs=ref_crs,
            transform=ref_transform,
            count=src.count,
            dtype=src.dtypes[0],
            compress="lzw"
        )

        # Allocate destination array (bands, h, w)
        resampled = np.empty((src.count, ref_height, ref_width), dtype=src.dtypes[0])

        # Reproject all bands
        reproject(
            source=rasterio.band(src, list(range(1, src.count + 1))),
            destination=resampled,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=ref_transform,
            dst_crs=ref_crs,
            resampling=Resampling.bilinear
        )

        # Write resampled result
        with rasterio.open(out_file, "w", **profile) as dst:
            dst.write(resampled)

        print(f"✅ Resampled {os.path.basename(tif_file)} → {out_file} ({src.count} bands preserved)")

✅ Resampled ERA5_d2m_2015_2016_stack.tif → C:\Users\Ankit\Datasets_Forest_fire\ERA5_resampled_to_VIIRS\ERA5_d2m_2015_2016_stack_toVIIRS.tif (17544 bands preserved)
✅ Resampled ERA5_t2m_2015_2016_stack.tif → C:\Users\Ankit\Datasets_Forest_fire\ERA5_resampled_to_VIIRS\ERA5_t2m_2015_2016_stack_toVIIRS.tif (17544 bands preserved)
✅ Resampled ERA5_tp_2015_2016_stack.tif → C:\Users\Ankit\Datasets_Forest_fire\ERA5_resampled_to_VIIRS\ERA5_tp_2015_2016_stack_toVIIRS.tif (17544 bands preserved)
✅ Resampled ERA5_u10_2015_2016_stack.tif → C:\Users\Ankit\Datasets_Forest_fire\ERA5_resampled_to_VIIRS\ERA5_u10_2015_2016_stack_toVIIRS.tif (17544 bands preserved)
✅ Resampled ERA5_v10_2015_2016_stack.tif → C:\Users\Ankit\Datasets_Forest_fire\ERA5_resampled_to_VIIRS\ERA5_v10_2015_2016_stack_toVIIRS.tif (17544 bands preserved)


In [5]:
import os
import rasterio

old_path = r"C:\Users\Ankit\Datasets_Forest_fire\ERA5_fast_tif_stacks\ERA5_d2m_2015_2016_stack.tif"
new_path = r"C:\Users\Ankit\Datasets_Forest_fire\ERA5_resampled_to_VIIRS\ERA5_d2m_2015_2016_stack_toVIIRS.tif"

def info(p):
    with rasterio.open(p) as ds:
        tags = ds.tags(ns='IMAGE_STRUCTURE')
        return {
            "path": p,
            "size_mb": os.path.getsize(p)/1024/1024,
            "bands": ds.count,
            "shape": (ds.height, ds.width),
            "dtype": ds.dtypes[0],
            "crs": str(ds.crs),
            "compress": tags.get("COMPRESSION", "None"),
            "blocksize": (tags.get("BLOCKXSIZE"), tags.get("BLOCKYSIZE")),
        }

print(info(old_path))
print(info(new_path))


{'path': 'C:\\Users\\Ankit\\Datasets_Forest_fire\\ERA5_fast_tif_stacks\\ERA5_d2m_2015_2016_stack.tif', 'size_mb': 744.0819244384766, 'bands': 17544, 'shape': (13, 17), 'dtype': 'float32', 'crs': 'EPSG:4326', 'compress': 'DEFLATE', 'blocksize': (None, None)}
{'path': 'C:\\Users\\Ankit\\Datasets_Forest_fire\\ERA5_resampled_to_VIIRS\\ERA5_d2m_2015_2016_stack_toVIIRS.tif', 'size_mb': 16.261449813842773, 'bands': 17544, 'shape': (13, 17), 'dtype': 'float32', 'crs': 'EPSG:4326', 'compress': 'LZW', 'blocksize': (None, None)}
